In [ ]:
import re
import cv2
import glob
import pytesseract
import pandas as pd
from time import sleep
from selenium import webdriver

PATH = "C:/Program Files (x86)/chromedriver.exe"
pytesseract.pytesseract.tesseract_cmd = <local_tesseract_file_path>

imagelink = <image_file_path>

In [ ]:
link = imagelink + "example_1.jpg"

image = cv2.imread(link, 0)

data = pytesseract.image_to_string(image, lang='eng',config='--psm 6')

print(data)

In [ ]:
link = imagelink + "example_2.jpg"

image = cv2.imread(link, 0)

data = pytesseract.image_to_string(image, lang='eng',config='--psm 6')

print(data)